## Test for pytorch

#### Import necessary libraries

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from DeepEverest import DeepEverest

In [2]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=False)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, inputs):
        return self.model(inputs)

In [4]:
dataset = torch.tensor([])
dataset_labels = torch.tensor([])
for num, (data, data_set) in enumerate(test_loader):
    if num == 0:
        dataset = torch.tensor(data)
        dataset_labels = torch.tensor(data_set)
    else:
        dataset = torch.cat((dataset, data), 0)
        dataset_labels = torch.cat((dataset_labels, data_set), 0)
model = Net()
lib_file = "index/build/lib.linux-x86_64-3.8/deepeverst_index.cpython-38-x86_64-linux-gnu.so"
model = DeepEverest(model, True, lib_file, dataset)

/tmp/ipykernel_159073/2494629665.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = torch.tensor(data)
/tmp/ipykernel_159073/2494629665.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_labels = torch.tensor(data_set)


#### Construct a new pytorch network, and construct the network into the DeepEverest API class

In [5]:
model = Net()
model = DeepEverest(model, True, lib_file, dataset)

#### Load a pretrained weight as an example

In [6]:
model.load_weights('mnist.pth')

#### print out all the layer outputs

In [7]:
model.get_layer_outputs()

{'': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42a90>,
 'model': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42790>,
 'model.0': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42e50>,
 'model.1': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42af0>,
 'model.2': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42be0>,
 'model.3': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42700>,
 'model.4': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42c10>,
 'model.5': <torch.utils.hooks.RemovableHandle at 0x7f70d1a42c40>,
 'model.6': <torch.utils.hooks.RemovableHandle at 0x7f720de74b80>,
 'model.7': <torch.utils.hooks.RemovableHandle at 0x7f720de74610>,
 'model.8': <torch.utils.hooks.RemovableHandle at 0x7f720de74b20>,
 'model.9': <torch.utils.hooks.RemovableHandle at 0x7f720de74df0>,
 'model.10': <torch.utils.hooks.RemovableHandle at 0x7f720de74850>,
 'model.11': <torch.utils.hooks.RemovableHandle at 0x7f720de74910>,
 'model.12': <torch.utils.hooks.RemovableHandle at 0x7f720de74a60>}

In [8]:
dataset_labels

tensor([7, 2, 1,  ..., 4, 5, 6])

In [9]:
for name, module in model.get_model().named_modules():
    print(name)


model
model.0
model.1
model.2
model.3
model.4
model.5
model.6
model.7
model.8
model.9
model.10
model.11
model.12


#### Choose one of the layer names

In [10]:
all_layer_names = model.get_all_layer_names()
print(all_layer_names)
layer_name = "model.2"
layer_id = all_layer_names.index(layer_name)
print(layer_id)

['', 'model', 'model.0', 'model.1', 'model.2', 'model.3', 'model.4', 'model.5', 'model.6', 'model.7', 'model.8', 'model.9', 'model.10', 'model.11', 'model.12']
4


In [11]:
layer_result = model.get_layer_results_by_layer_id(layer_id)

In [12]:
print(layer_result.shape)

(10000, 10, 12, 12)


#### Construct the index before doing the query

In [13]:
model.construct_index(layer_id)

#### Give top-k activation neurons of image 659 as an example

In [14]:
topk_activations = model.get_topk_activations_given_images([659], layer_name, 20)
print(topk_activations)

[[(15.056664, (4, 4, 6)), (14.274166, (4, 5, 6)), (13.664486, (4, 6, 6)), (13.554054, (4, 3, 6)), (13.502932, (4, 8, 6)), (13.477571, (4, 9, 6)), (13.340709, (1, 11, 6)), (13.27995, (4, 4, 7)), (13.197538, (1, 5, 3)), (13.188957, (4, 3, 7)), (13.090613, (1, 10, 6)), (12.774042, (1, 5, 4)), (12.733601, (4, 2, 6)), (12.522223, (4, 7, 6)), (12.089676, (4, 5, 7)), (11.97962, (1, 11, 7)), (11.603155, (4, 8, 5)), (11.496715, (4, 2, 7)), (11.474282, (0, 3, 7)), (11.457584, (1, 4, 4))]]


In [15]:
from NeuronGroup import *
image_sample_id = 659
topk_activations_neurons = [x[1] for x in topk_activations[0]]
print(topk_activations_neurons)
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=topk_activations_neurons[:3])

[(4, 4, 6), (4, 5, 6), (4, 6, 6), (4, 3, 6), (4, 8, 6), (4, 9, 6), (1, 11, 6), (4, 4, 7), (1, 5, 3), (4, 3, 7), (1, 10, 6), (1, 5, 4), (4, 2, 6), (4, 7, 6), (4, 5, 7), (1, 11, 7), (4, 8, 5), (4, 2, 7), (0, 3, 7), (1, 4, 4)]


#### Query the top 20 closest images with respect to the top 3 neurons found in the last step

In [16]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 0.10146905481815338, max in answer: 1.40303635597229, images run: 67
threshold: 0.20118562877178192, max in answer: 1.1438370943069458, images run: 132
threshold: 0.2788737714290619, max in answer: 0.9198803901672363, images run: 196
threshold: 0.42958271503448486, max in answer: 0.7807649374008179, images run: 260
threshold: 0.5854843854904175, max in answer: 0.7519282102584839, images run: 325
threshold: 0.7778107523918152, max in answer: 0.7440550923347473, images run: 389
======================= NTA exited =======================


#### Print out distance and corresponding image indices

In [17]:
print(top_k)
print(exit_msg)

[(-0.7440551, 2915), (-0.7409354, 6074), (-0.6970588, 8109), (-0.6478751, 7822), (-0.6055464, 3611), (-0.58405477, 6118), (-0.5718313, 4011), (-0.53731173, 8100), (-0.52412087, 4097), (-0.5215469, 4861), (-0.46935377, 6988), (-0.44762346, 6096), (-0.4275496, 8048), (-0.4268946, 7783), (-0.42228025, 1909), (-0.41333342, 3229), (-0.3712188, 1480), (-0.23766346, 8679), (-0.113409005, 8112), (0.0, 659)]
termination: images run: 389


In [18]:
image_ids = [1, 0, 321, 810]
image_ids

[1, 0, 321, 810]

#### This tool is capable of search group of images in one query

In [19]:
topk_activations = model.get_topk_activations_given_images(image_ids, layer_name, 20)

In [20]:
topk_activations

[[(15.404506, (1, 10, 6)),
  (14.740844, (1, 10, 9)),
  (14.725686, (1, 10, 5)),
  (14.658377, (1, 10, 4)),
  (14.074226, (1, 10, 7)),
  (14.018891, (1, 10, 10)),
  (13.745972, (1, 10, 3)),
  (13.607746, (1, 2, 5)),
  (13.291188, (1, 10, 8)),
  (12.143644, (1, 2, 4)),
  (11.776539, (4, 2, 6)),
  (11.771938, (4, 6, 4)),
  (11.722886, (4, 8, 3)),
  (11.716561, (1, 1, 5)),
  (11.585614, (4, 3, 6)),
  (11.531162, (4, 7, 3)),
  (11.222839, (4, 5, 4)),
  (11.1286335, (4, 4, 5)),
  (11.101066, (1, 10, 11)),
  (11.002071, (4, 5, 5))],
 [(14.003622, (1, 4, 4)),
  (13.753479, (1, 4, 5)),
  (13.233683, (1, 4, 3)),
  (13.210785, (1, 4, 6)),
  (12.756104, (1, 4, 7)),
  (10.772835, (4, 11, 4)),
  (10.763884, (4, 10, 5)),
  (10.652996, (4, 4, 8)),
  (10.51455, (1, 3, 3)),
  (10.362712, (1, 4, 2)),
  (10.331228, (1, 3, 2)),
  (9.836474, (4, 9, 5)),
  (9.813205, (4, 5, 8)),
  (9.546487, (4, 10, 4)),
  (9.501915, (4, 8, 6)),
  (9.265276, (4, 11, 5)),
  (9.024993, (4, 7, 7)),
  (8.871638, (4, 6, 7)),
  (

#### A group of neurons can be specified by the user to do the query

In [21]:
image_sample_id = 810
neuron_list = [(1, 4, 4), (1, 4, 3), (1, 4, 5)]
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=neuron_list)

#### Find the closest image with respect o the neuron group specified

In [22]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)
print(top_k)

image 810, size of neuron group 3
threshold: 0.4012274742126465, max in answer: 1.244203805923462, images run: 67
threshold: 0.7033581137657166, max in answer: 1.01547372341156, images run: 131
threshold: 0.9670814871788025, max in answer: 0.9958177804946899, images run: 195
threshold: 1.2530744075775146, max in answer: 0.9958177804946899, images run: 260
======================= NTA exited =======================
[(-0.9958178, 9990), (-0.9594661, 8555), (-0.9563361, 6316), (-0.92077124, 6050), (-0.88487566, 6158), (-0.86679006, 5179), (-0.85618293, 3743), (-0.84050524, 60), (-0.83489597, 1935), (-0.81305367, 7960), (-0.78954655, 2187), (-0.73703915, 6366), (-0.6998202, 5326), (-0.6811, 4865), (-0.6809903, 6006), (-0.65254635, 4064), (-0.6416337, 7650), (-0.6308938, 3125), (-0.57279134, 822), (0.0, 810)]


In [23]:
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=[(5, 0, 2), (8, 2, 2), (2, 4, 1)])

In [24]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)
print(top_k)

image 810, size of neuron group 3
threshold: 0.03818821907043457, max in answer: 0.1384831666946411, images run: 427
threshold: 0.20468735694885254, max in answer: 0.07035708427429199, images run: 851
======================= NTA exited =======================
[(-0.070357084, 4176), (-0.0684191, 4659), (-0.06708133, 3394), (-0.06519842, 9109), (-0.05986345, 2700), (-0.057710648, 554), (-0.052951217, 4042), (-0.045683503, 1100), (-0.04501462, 6053), (-0.04026103, 9618), (-0.039102912, 4421), (-0.03840089, 738), (-0.03818822, 812), (-0.03238964, 3431), (-0.026446223, 362), (-0.025786996, 924), (-0.024932742, 6871), (-0.020506978, 6075), (-0.0010017157, 2115), (0.0, 810)]


In [25]:
print(model.get_current_index_layer_id())

4


#### Can construct indices for multiple layers, and the index constructed so far can be saved to disk

In [26]:
model.save_index_map("saved_map")

In [27]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)
print(top_k)

image 810, size of neuron group 3
threshold: 0.03818821907043457, max in answer: 0.1384831666946411, images run: 427
threshold: 0.20468735694885254, max in answer: 0.07035708427429199, images run: 851
======================= NTA exited =======================
[(-0.070357084, 4176), (-0.0684191, 4659), (-0.06708133, 3394), (-0.06519842, 9109), (-0.05986345, 2700), (-0.057710648, 554), (-0.052951217, 4042), (-0.045683503, 1100), (-0.04501462, 6053), (-0.04026103, 9618), (-0.039102912, 4421), (-0.03840089, 738), (-0.03818822, 812), (-0.03238964, 3431), (-0.026446223, 362), (-0.025786996, 924), (-0.024932742, 6871), (-0.020506978, 6075), (-0.0010017157, 2115), (0.0, 810)]


#### Saved indices can be reload back

In [28]:
model.load_index_map("saved_map")

## Test for tensorflow

#### This part is aimed to show that the code also works with Tensorflow model, the process is similar, and user should create a super class shown in MnistVGG_api class, and use this class to do queries. The procedure is very similar comparing to using a Pytorch model. 

In [29]:
import os
import numpy as np
from utils import load_mnist_vgg_dataset_model
from models.MnistVGG_api import MnistVGG

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [30]:
x_train, y_train, x_test, y_test = load_mnist_vgg_dataset_model()
dataset = x_test
dataset_labels = y_test
del x_train
del y_train
model = MnistVGG(lib_file, dataset)
all_layer_names = model.get_all_layer_names()
print(all_layer_names)

2022-07-21 01:46:56.988340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 01:46:56.996872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-07-21 01:46:56.996896: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-21 01:46:56.997495: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

['input_1', 'zero_padding2d', 'conv2d', 'batch_normalization', 'activation', 'conv2d_1', 'batch_normalization_1', 'activation_1', 'max_pooling2d', 'conv2d_2', 'batch_normalization_2', 'activation_2', 'conv2d_3', 'batch_normalization_3', 'activation_3', 'max_pooling2d_1', 'conv2d_4', 'batch_normalization_4', 'activation_4', 'conv2d_5', 'batch_normalization_5', 'activation_5', 'conv2d_6', 'batch_normalization_6', 'activation_6', 'max_pooling2d_2', 'conv2d_7', 'batch_normalization_7', 'activation_7', 'conv2d_8', 'batch_normalization_8', 'activation_8', 'conv2d_9', 'batch_normalization_9', 'activation_9', 'max_pooling2d_3', 'conv2d_10', 'batch_normalization_10', 'activation_10', 'conv2d_11', 'batch_normalization_11', 'activation_11', 'conv2d_12', 'batch_normalization_12', 'activation_12', 'max_pooling2d_4', 'flatten', 'dense', 'dropout', 'dense_1', 'dropout_1', 'dense_2', 'activation_13']


In [31]:
layer_name = "activation_12"
layer_id = all_layer_names.index(layer_name)

In [32]:
print(dataset.shape)
print(dataset_labels.shape)

(10000, 28, 28, 1)
(10000, 10)


In [33]:
layer_result = model.get_layer_results_by_layer_id(layer_id)
print(layer_result.shape)
print(type(layer_result))

(10000, 2, 2, 512)
<class 'numpy.ndarray'>


In [34]:
model.construct_index(layer_id)

In [35]:
topk_activations = model.get_topk_activations_given_images([659], layer_name, 20)
print(topk_activations)

[[(2.2470708, (1, 1, 374)), (1.9390898, (0, 0, 113)), (1.9011347, (0, 0, 358)), (1.8457984, (1, 0, 335)), (1.8241947, (1, 1, 62)), (1.7726848, (1, 0, 182)), (1.7536622, (1, 1, 129)), (1.7496964, (0, 0, 95)), (1.7491626, (1, 0, 441)), (1.6753774, (1, 1, 177)), (1.6667684, (0, 0, 46)), (1.6587316, (0, 1, 340)), (1.6440474, (0, 1, 285)), (1.6398696, (1, 1, 428)), (1.6205437, (0, 1, 365)), (1.6141273, (0, 1, 461)), (1.5758135, (0, 0, 245)), (1.558946, (0, 0, 45)), (1.5521429, (0, 1, 336)), (1.5128994, (0, 1, 379))]]


In [36]:
from NeuronGroup import *
image_sample_id = 659
topk_activations_neurons = [x[1] for x in topk_activations[0]]
print(topk_activations_neurons)
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=topk_activations_neurons[:3])

[(1, 1, 374), (0, 0, 113), (0, 0, 358), (1, 0, 335), (1, 1, 62), (1, 0, 182), (1, 1, 129), (0, 0, 95), (1, 0, 441), (1, 1, 177), (0, 0, 46), (0, 1, 340), (0, 1, 285), (1, 1, 428), (0, 1, 365), (0, 1, 461), (0, 0, 245), (0, 0, 45), (0, 1, 336), (0, 1, 379)]


In [37]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 0.009084991179406643, max in answer: 1.0652904510498047, images run: 446
threshold: 0.2361185997724533, max in answer: 0.7390391230583191, images run: 879
threshold: 0.25425785779953003, max in answer: 0.7369704842567444, images run: 1320
threshold: 0.4669073820114136, max in answer: 0.648984968662262, images run: 1707
threshold: 0.5761292576789856, max in answer: 0.648984968662262, images run: 2125
threshold: 0.7417832016944885, max in answer: 0.648984968662262, images run: 2537
======================= NTA exited =======================


In [38]:
print(top_k)
print(exit_msg)

[(-0.64898497, 8043), (-0.6488453, 3073), (-0.63240474, 2915), (-0.6309901, 9505), (-0.6308845, 7985), (-0.61490285, 9664), (-0.57323945, 1941), (-0.57080907, 9637), (-0.5610424, 1903), (-0.5584375, 3166), (-0.5519714, 468), (-0.536307, 5649), (-0.5042956, 667), (-0.3780876, 7902), (-0.37677968, 3055), (-0.37317902, 5655), (-0.3371267, 4176), (-0.32671073, 4199), (-0.15818696, 3808), (0.0, 659)]
termination: images run: 2537


In [39]:
print(model.get_current_index_layer_id())

44


In [40]:
model.save_index_map("saved_map")

In [41]:
model.load_index_map("saved_map")

In [42]:
top_k, exit_msg = model.answer_query_with_guarantee(image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 0.009084991179406643, max in answer: 1.0652904510498047, images run: 446
threshold: 0.2361185997724533, max in answer: 0.7390391230583191, images run: 879
threshold: 0.25425785779953003, max in answer: 0.7369704842567444, images run: 1320
threshold: 0.4669073820114136, max in answer: 0.648984968662262, images run: 1707
threshold: 0.5761292576789856, max in answer: 0.648984968662262, images run: 2125
threshold: 0.7417832016944885, max in answer: 0.648984968662262, images run: 2537
======================= NTA exited =======================


In [43]:
print(top_k)
print(exit_msg)

[(-0.64898497, 8043), (-0.6488453, 3073), (-0.63240474, 2915), (-0.6309901, 9505), (-0.6308845, 7985), (-0.61490285, 9664), (-0.57323945, 1941), (-0.57080907, 9637), (-0.5610424, 1903), (-0.5584375, 3166), (-0.5519714, 468), (-0.536307, 5649), (-0.5042956, 667), (-0.3780876, 7902), (-0.37677968, 3055), (-0.37317902, 5655), (-0.3371267, 4176), (-0.32671073, 4199), (-0.15818696, 3808), (0.0, 659)]
termination: images run: 2537


In [44]:
layer_name = "activation_10"
layer_id = all_layer_names.index(layer_name)

In [45]:
layer_result = model.get_layer_results_by_layer_id(layer_id)
print(layer_result.shape)
print(type(layer_result))

(10000, 2, 2, 512)
<class 'numpy.ndarray'>


In [46]:
model.construct_index(layer_id)

In [47]:
model.get_current_index_layer_id()

38

In [48]:
model.construct_index(44)